In [1]:
import pytorch_lightning as pl
from matplotlib import pyplot as plt
import seaborn as sns
from cca_zoo.deepmodels import (
    DCCA,
    DCCA_NOI,
    DCCA_SDL,
    BarlowTwins,
    get_dataloaders,
    
)
from cca_zoo.deepmodels.utils import architectures, objectives
from cca_zoo.plotting import pairplot_label
from cca_zoo.data import CCA_Dataset
from cca_zoo.models import CCA

from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np


os.chdir('../raw_data')


In [3]:
os.listdir()

['ALL_GNPS.json',
 'ALL_GNPS_15_12_2021_positive_annotated.pickle',
 'ALL_GNPS_15_12_2021_positive_annotated_classifications.txt',
 'all_ms2ds_embedding.pickle',
 'all_spec_embedding.pickle',
 'df_ALL_GNPS_15_12_2021_positive_annotated_embeddings_class.pickle',
 'df_classes_all_embeddings.pickle',
 'df_classes_max3_embeddings.pickle',
 'df_classes_max5_embeddings.pickle',
 'df_classes_unique_embeddings.pickle',
 'GNPS_15_12_2021_ms2ds_embeddings.pickle',
 'lightning_logs',
 'max3_mol2vec_embedding.pickle',
 'max3_ms2ds_embedding.pickle',
 'max3_spec_embedding.pickle',
 'max5_mol2vec_embedding.pickle',
 'max5_ms2ds_embedding.pickle',
 'max5_spec_embedding.pickle',
 'model_300dim.pkl',
 'mol2vec_embedding.pickle',
 'ms2ds_embeddings_unique_GNPS_15_12_2021.pickle',
 'ms2ds_model_GNPS_15_12_2021.hdf5',
 'ms2ds_pred_euc',
 'plot_validation.py',
 'spec2vec_embeddings_unique_GNPS_15_12_2021.pickle',
 'spec2vec_model_GNPS_15_12_2021.model',
 'spec2vec_model_GNPS_15_12_2021.model.syn1neg.npy',


#### Functions to load and or write to pickle files

In [2]:
class Files:
    def __init__(self, filename):
        self.file = filename
        
    def write_to_file(self, data):
        with open(self.file, 'wb') as f:
            pickle.dump(data, f) 
        return None
    
    def load_pickle(self):
        data = pd.read_pickle(self.file)
        return data
    
    def load_csv(self, sep, usecols=None):
        data = pd.read_csv(self.file, sep=sep, usecols=usecols)
        return data
        
        
    

In [117]:
# load df with all info
path = './df_classes_max3_embeddings.pickle'
df_all = Files(path).load_pickle()#[:2000] #just a subset for scripting
df_all.shape

(41447, 8)

In [118]:
#split into train and test dataset
train_df, test_df = \
train_test_split(df_all, test_size=0.3, random_state=42)#, stratify=df_all['cf_class'])

#Split train dataset into train and validation set
test_df, val_df = train_test_split(test_df, test_size=0.1, random_state=42)


#extract the 2 view, v1 == spectra embeddings, v2==structure embeddings
v1_train, v1_test = np.array([x for x in train_df['ms2ds']]), np.array([x for x in test_df['ms2ds']])
v2_train, v2_test = np.array([x for x in train_df['mol2vec']]), np.array([x for x in test_df['mol2vec']])

# validation
v1_val, v2_val = np.array([x for x in val_df['ms2ds']]), np.array([x for x in val_df['mol2vec']])

In [119]:
# What does CCA dataset do????
train_dataset = CCA_Dataset([v1_train, v2_train])
test_dataset = CCA_Dataset([v1_test, v2_test])
val_dataset = CCA_Dataset([v1_val, v2_val])

train_loader , val_loader = get_dataloaders(train_dataset, val_dataset, batch_size=128, num_workers=6,
                                              drop_last=False)
test_loader = get_dataloaders(test_dataset,batch_size=128, num_workers=6,
                                              drop_last=False)

In [164]:
#params
N = len(train_dataset)
latent_dims = 20
epochs = 100
cca = CCA(latent_dims=latent_dims).fit((v1_train, v2_train))

# define encoders
encoder_1 = architectures.Encoder(latent_dims=20, feature_size=v1_train.shape[1],layer_sizes=(128,))
                                     
encoder_2 = architectures.Encoder(latent_dims=20, feature_size=v2_train.shape[1], layer_sizes=(128,))

# define variant algorithmns
# 1. DCCA
dcca = DCCA(
        latent_dims=latent_dims,
        encoders=[encoder_1, encoder_2],
        objective=objectives.CCA,
        )

# 2. SDL
sdl = DCCA_SDL(latent_dims,
                N=N, encoders=[encoder_1, encoder_2],
                lam=1e-2, lr=1e-3)

# Define the trainer
trainer = pl.Trainer(
        max_epochs=epochs, log_every_n_steps=1,
        callbacks=[pl.callbacks.early_stopping.EarlyStopping(monitor="val/corr")])# early stopage to reduce overfitting

# train
trainer.fit(dcca, train_loader,val_loader)

#transform the views to get z projections


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type       | Params
----------------------------------------
0 | encoders | ModuleList | 69.4 K
----------------------------------------
69.4 K    Trainable params
0         Non-trainable params
69.4 K    Total params
0.278     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

_LinAlgError: torch.linalg.eigh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 19).

In [148]:
cca_train_corr = cca.score([v1_train, v2_train])
cca_test_corr = cca.score([v1_test, v2_test])

In [130]:
dcca_corr = dcca.score(train_loader)
dcca_corr

array([0.96029372, 0.94371559, 0.92509154, 0.88447767, 0.87225474,
       0.86303599, 0.84117505, 0.82341388, 0.8175819 , 0.80031464])

In [131]:
dcca_test_corr = dcca.score(test_loader)
dcca_test_corr

array([0.94739839, 0.92042063, 0.91469235, 0.85021026, 0.81888865,
       0.80710917, 0.80327086, 0.79571386, 0.78087832, 0.76230672])

In [149]:
sdl_train_corr = sdl.score(train_loader)
sdl_train_corr[0:5]

array([0.61873624, 0.601584  , 0.48265693, 0.34426731, 0.30904485])

In [150]:
sdl_test_corr = sdl.score(test_loader)
sdl_test_corr[0:5]

array([0.62638568, 0.5828956 , 0.47934977, 0.35443136, 0.31211721])

In [ ]:
# plot train correlation vs test correlation
x=[x for x in range(latent_dims)]
fig, ax = plt.subplots()
sns.set_style('whitegrid')
dfs = [[dcca_corr,dcca_test_corr],[cca_train_corr, cca_test_corr],\
       [sdl_train_corr,sdl_test_corr]]
for df in dfs:
    sns.lineplot(x=x,y=df[0],ax=ax)
    sns.lineplot(x=x,y=df[1],ax=ax)
plt.legend(labels=['DCCA Train', 'DCCA Test',\
                  'CCA Train', 'CCA Test',\
                  'SDL Train', 'SDL Test'], loc=(1.04, 0.5), fontsize=15)
plt.xlabel('Latent Dimensions', fontsize=15)
plt.ylabel('Correlation', fontsize=15)
plt.ylim(0, 1)
#plt.legend(loc=(1.04, 0))
plt.xlim(0, latent_dims)
plt.yticks(np.arange(0, 1, step=0.1))
plt.show()

In [175]:
#score test dataset
# check the validation loop warning

In [78]:
v1_train.shape

(11200, 200)